In [1]:
%load_ext rpy2.ipython

In [3]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb

Attaching pack

In [4]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"
N_FOLDS <- 10

In [5]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030_time_cross_20.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 26 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(26): SRR3593523 SRR7817700 ... SRR7817644 SRR3593580
colData names(3): Sample Dose sizeFactor


In [6]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [7]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 26
                   SRR3593523 SRR7817700 SRR7817648 SRR7817672 SRR7817662
ENSMUSG00000000001  14.547886  15.798199  15.281283  15.275252  15.736033
ENSMUSG00000000028   6.923071   7.650167   6.913712   7.198065   7.191227
ENSMUSG00000000049  20.789011  20.246032  20.562958  20.083925  20.463679
ENSMUSG00000000058   8.153123   7.727127   7.585587   7.994352   7.792455
ENSMUSG00000000085  10.513756   9.724836  10.184706  10.061571   9.490532
ENSMUSG00000000088  15.092601  16.822697  17.012867  15.960130  16.829317
                   SRR7817616 SRR4317609 SRR1636591 SRR1636586 SRR7817695
ENSMUSG00000000001  15.546543  15.071420  15.875510  16.383344  15.748492
ENSMUSG00000000028   6.973028   7.586469   7.500169   7.748654   6.350478
ENSMUSG00000000049  20.424124  20.638897  20.637437  20.514960  20.140578
ENSMUSG00000000058   8.400943   8.904841   9.002105   9.105034   7.965077
ENSMUSG00000000085   9.715885   9.469542   9.927091   9.370924  10.223300
ENSMUSG00000000088  16.369556  

In [8]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030_time_cross_20.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0 30 30 30 30 30 30 30 30 30 30 30 30
[26] 30


In [9]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030_time_cross_20.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

 [1] "ENSMUSG00000041229" "ENSMUSG00000060093" "ENSMUSG00000060639"
 [4] "ENSMUSG00000060678" "ENSMUSG00000060981" "ENSMUSG00000061482"
 [7] "ENSMUSG00000064288" "ENSMUSG00000067455" "ENSMUSG00000069266"
[10] "ENSMUSG00000069274" "ENSMUSG00000069305" "ENSMUSG00000069306"
[13] "ENSMUSG00000091405" "ENSMUSG00000096010"

$`R-MMU-1296045`
[1] "ENSMUSG00000041248" "ENSMUSG00000044708" "ENSMUSG00000051497"

$`R-MMU-383190`
[1] "ENSMUSG00000041301"

$`R-MMU-5678863`
[1] "ENSMUSG00000041301"

$`R-MMU-8866851`
[1] "ENSMUSG00000041301"

$`R-MMU-375776`
[1] "ENSMUSG00000041313"

$`R-MMU-5218853`
[1] "ENSMUSG00000041417" "ENSMUSG00000062960"

$`R-MMU-9607226`
[1] "ENSMUSG00000041417" "ENSMUSG00000059923" "ENSMUSG00000110206"

$`R-MMU-70881`
[1] "ENSMUSG00000041426"

$`R-MMU-110224`
[1] "ENSMUSG00000041429"

$`R-MMU-110226`
[1] "ENSMUSG00000041429"

$`R-MMU-110227`
[1] "ENSMUSG00000041429"

$`R-MMU-110229`
[1] "ENSMUSG00000041429"

$`R-MMU-9014052`
[1] "ENSMUSG00000041488" "ENSMUSG00000053841"

$`R

[19] "ENSMUSG00000094818"

$`R-MMU-1467269`
 [1] "ENSMUSG00000044748" "ENSMUSG00000044863" "ENSMUSG00000046354"
 [4] "ENSMUSG00000054763" "ENSMUSG00000059230" "ENSMUSG00000073735"
 [7] "ENSMUSG00000074678" "ENSMUSG00000075571" "ENSMUSG00000075572"
[10] "ENSMUSG00000075573" "ENSMUSG00000075574"

$`R-MMU-1471322`
 [1] "ENSMUSG00000044748" "ENSMUSG00000044863" "ENSMUSG00000046354"
 [4] "ENSMUSG00000054763" "ENSMUSG00000059230" "ENSMUSG00000073735"
 [7] "ENSMUSG00000074678" "ENSMUSG00000075571" "ENSMUSG00000075572"
[10] "ENSMUSG00000075573" "ENSMUSG00000075574"

$`R-MMU-5638141`
 [1] "ENSMUSG00000044791" "ENSMUSG00000069265" "ENSMUSG00000069267"
 [4] "ENSMUSG00000069273" "ENSMUSG00000069310" "ENSMUSG00000074403"
 [7] "ENSMUSG00000081058" "ENSMUSG00000093769" "ENSMUSG00000099517"
[10] "ENSMUSG00000099583" "ENSMUSG00000100210" "ENSMUSG00000101355"
[13] "ENSMUSG00000101972"

$`R-MMU-964773`
[1] "ENSMUSG00000044819"

$`R-MMU-9013198`
[1] "ENSMUSG00000044986"

$`R-MMU-428231`
[1] "ENSMUSG000000

 [7] "ENSMUSG00000064288" "ENSMUSG00000067455" "ENSMUSG00000069266"
[10] "ENSMUSG00000069274" "ENSMUSG00000069305" "ENSMUSG00000069306"
[13] "ENSMUSG00000091405" "ENSMUSG00000096010"

$`R-MMU-6805740`
[1] "ENSMUSG00000049327" "ENSMUSG00000059552"

$`R-MMU-428585`
[1] "ENSMUSG00000049624"

$`R-MMU-9844587`
[1] "ENSMUSG00000049721"

$`R-MMU-1614544`
[1] "ENSMUSG00000049858"

$`R-MMU-742345`
[1] "ENSMUSG00000049922"

$`R-MMU-417820`
[1] "ENSMUSG00000049929"

$`R-MMU-9707606`
[1] "ENSMUSG00000049940"

$`R-MMU-9707683`
[1] "ENSMUSG00000049940"

$`R-MMU-9707856`
[1] "ENSMUSG00000049940"

$`R-MMU-1483087`
[1] "ENSMUSG00000050017"

$`R-MMU-1483211`
[1] "ENSMUSG00000050017"

$`R-MMU-1483219`
[1] "ENSMUSG00000050017"

$`R-MMU-1483229`
[1] "ENSMUSG00000050017"

$`R-MMU-5693691`
[1] "ENSMUSG00000050097" "ENSMUSG00000056973"

$`R-MMU-9749647`
[1] "ENSMUSG00000050097"

$`R-MMU-9749792`
[1] "ENSMUSG00000050097" "ENSMUSG00000056973"

$`R-MMU-5696119`
[1] "ENSMUSG00000050103"

$`R-MMU-9673827`
[1] "ENS

[1] "ENSMUSG00000055978"

$`R-MMU-9603983`
[1] "ENSMUSG00000055978"

$`R-MMU-168412`
[1] "ENSMUSG00000055994"

$`R-MMU-708349`
[1] "ENSMUSG00000055994"

$`R-MMU-6798474`
[1] "ENSMUSG00000056054" "ENSMUSG00000056071"

$`R-MMU-6798528`
[1] "ENSMUSG00000056054" "ENSMUSG00000056071"

$`R-MMU-9844860`
[1] "ENSMUSG00000056091"

$`R-MMU-446185`
[1] "ENSMUSG00000056131"

$`R-MMU-9706330`
[1] "ENSMUSG00000056153" "ENSMUSG00000110206"

$`R-MMU-8944218`
[1] "ENSMUSG00000056174" "ENSMUSG00000068196"

$`R-MMU-111881`
[1] "ENSMUSG00000056220"

$`R-MMU-111883`
[1] "ENSMUSG00000056220"

$`R-MMU-111898`
[1] "ENSMUSG00000056220" "ENSMUSG00000063358"

$`R-MMU-1482759`
[1] "ENSMUSG00000056220"

$`R-MMU-6791016`
[1] "ENSMUSG00000056370"

$`R-MMU-9624789`
[1] "ENSMUSG00000056370"

$`R-MMU-5358592`
[1] "ENSMUSG00000056394"

$`R-MMU-69173`
[1] "ENSMUSG00000056394"

$`R-MMU-9014847`
[1] "ENSMUSG00000056427"

$`R-MMU-3857329`
[1] "ENSMUSG00000056501" "ENSMUSG00000063065" "ENSMUSG00000063358"

$`R-MMU-3857336`
[

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

[1] 9825
   [1] "R-MMU-170666"  "R-MMU-170671"  "R-MMU-170674"  "R-MMU-170686" 
   [5] "R-MMU-380073"  "R-MMU-392187"  "R-MMU-392195"  "R-MMU-392202" 
   [9] "R-MMU-749454"  "R-MMU-749456"  "R-MMU-751001"  "R-MMU-8964229"
  [13] "R-MMU-8964242" "R-MMU-8964329" "R-MMU-8982012" "R-MMU-8982019"
  [17] "R-MMU-8982020" "R-MMU-9021596" "R-MMU-9021600" "R-MMU-9021601"
  [21] "R-MMU-176298"  "R-MMU-176318"  "R-MMU-68914"   "R-MMU-68916"  
  [25] "R-MMU-68917"   "R-MMU-68960"   "R-MMU-481009"  "R-MMU-9021068"
  [29] "R-MMU-9021079" "R-MMU-3108203" "R-MMU-3108209" "R-MMU-3927824"
  [33] "R-MMU-4551655" "R-MMU-4551727" "R-MMU-4570463" "R-MMU-4570499"
  [37] "R-MMU-8937989" "R-MMU-163214"  "R-MMU-9709406" "R-MMU-9838627"
  [41] "R-MMU-9839113" "R-MMU-193641"  "R-MMU-193646"  "R-MMU-193650" 
  [45] "R-MMU-193653"  "R-MMU-193656"  "R-MMU-193665"  "R-MMU-193668" 
  [49] "R-MMU-193669"  "R-MMU-193679"  "R-MMU-193682"  "R-MMU-193684" 
  [53] "R-MMU-193694"  "R-MMU-193695"  "R-MMU-193696"  "R-MMU-193700

 [461] "R-MMU-382054"  "R-MMU-382055"  "R-MMU-382056"  "R-MMU-382058" 
 [465] "R-MMU-389083"  "R-MMU-389086"  "R-MMU-481007"  "R-MMU-8864036"
 [469] "R-MMU-8865275" "R-MMU-8865276" "R-MMU-9674093" "R-MMU-8858369"
 [473] "R-MMU-1181149" "R-MMU-1181153" "R-MMU-1225898" "R-MMU-1549526"
 [477] "R-MMU-6803719" "R-MMU-202723"  "R-MMU-2327746" "R-MMU-2328037"
 [481] "R-MMU-2426259" "R-MMU-265424"  "R-MMU-374686"  "R-MMU-4088264"
 [485] "R-MMU-9624017" "R-MMU-975340"  "R-MMU-191790"  "R-MMU-3215385"
 [489] "R-MMU-3215406" "R-MMU-3215426" "R-MMU-3215448" "R-MMU-5205799"
 [493] "R-MMU-5205861" "R-MMU-5218952" "R-MMU-5661117" "R-MMU-418904" 
 [497] "R-MMU-164832"  "R-MMU-164834"  "R-MMU-164840"  "R-MMU-8949580"
 [501] "R-MMU-5626938" "R-MMU-8985343" "R-MMU-9690401" "R-MMU-9690410"
 [505] "R-MMU-9690416" "R-MMU-9690868" "R-MMU-9690898" "R-MMU-9692133"
 [509] "R-MMU-9692136" "R-MMU-9692137" "R-MMU-9692138" "R-MMU-9692139"
 [513] "R-MMU-9692145" "R-MMU-9692156" "R-MMU-9692157" "R-MMU-9692268"
 [517]

 [921] "R-MMU-4332358" "R-MMU-4332359" "R-MMU-4332363" "R-MMU-442749" 
 [925] "R-MMU-445797"  "R-MMU-445813"  "R-MMU-4551451" "R-MMU-4551465"
 [929] "R-MMU-482772"  "R-MMU-5626507" "R-MMU-5626549" "R-MMU-6786205"
 [933] "R-MMU-71541"   "R-MMU-71588"   "R-MMU-9005848" "R-MMU-9005872"
 [937] "R-MMU-9007015" "R-MMU-9619355" "R-MMU-9619376" "R-MMU-9653585"
 [941] "R-MMU-9654521" "R-MMU-9654523" "R-MMU-9654525" "R-MMU-9654533"
 [945] "R-MMU-75885"   "R-MMU-8849345" "R-MMU-427666"  "R-MMU-5336231"
 [949] "R-MMU-5336365" "R-MMU-5336369" "R-MMU-9818467" "R-MMU-9818487"
 [953] "R-MMU-420214"  "R-MMU-420265"  "R-MMU-9733292" "R-MMU-8933256"
 [957] "R-MMU-8933295" "R-MMU-6800198" "R-MMU-6800200" "R-MMU-200318" 
 [961] "R-MMU-2187309" "R-MMU-6810233" "R-MMU-6810234" "R-MMU-6810235"
 [965] "R-MMU-6810238" "R-MMU-6814549" "R-MMU-6814555" "R-MMU-6814559"
 [969] "R-MMU-6814885" "R-MMU-9023840" "R-MMU-9761027" "R-MMU-199032" 
 [973] "R-MMU-265422"  "R-MMU-376419"  "R-MMU-382096"  "R-MMU-5334854"
 [977]

[1381] "R-MMU-6807875" "R-MMU-6807877" "R-MMU-6809010" "R-MMU-6809011"
[1385] "R-MMU-5692770" "R-MMU-5692775" "R-MMU-1011598" "R-MMU-114542" 
[1389] "R-MMU-1433415" "R-MMU-2029451" "R-MMU-2029454" "R-MMU-2029456"
[1393] "R-MMU-2029465" "R-MMU-2029466" "R-MMU-2029467" "R-MMU-2029469"
[1397] "R-MMU-205039"  "R-MMU-205136"  "R-MMU-2130194" "R-MMU-2197690"
[1401] "R-MMU-2424476" "R-MMU-2730840" "R-MMU-2730856" "R-MMU-2730889"
[1405] "R-MMU-376123"  "R-MMU-3858475" "R-MMU-389788"  "R-MMU-3928619"
[1409] "R-MMU-3928620" "R-MMU-3928642" "R-MMU-399930"  "R-MMU-399939" 
[1413] "R-MMU-399952"  "R-MMU-400682"  "R-MMU-418856"  "R-MMU-421142" 
[1417] "R-MMU-421144"  "R-MMU-421147"  "R-MMU-421148"  "R-MMU-442273" 
[1421] "R-MMU-442291"  "R-MMU-442314"  "R-MMU-445064"  "R-MMU-445072" 
[1425] "R-MMU-451347"  "R-MMU-451366"  "R-MMU-5218827" "R-MMU-5218839"
[1429] "R-MMU-5218841" "R-MMU-5218850" "R-MMU-5357445" "R-MMU-5357472"
[1433] "R-MMU-5357483" "R-MMU-5623622" "R-MMU-5623632" "R-MMU-5623667"
[1437]

[1841] "R-MMU-2160932" "R-MMU-2160935" "R-MMU-2169046" "R-MMU-2395364"
[1845] "R-MMU-2731078" "R-MMU-442497"  "R-MMU-8870732" "R-MMU-8878117"
[1849] "R-MMU-8937814" "R-MMU-8951966" "R-MMU-9821366" "R-MMU-445448" 
[1853] "R-MMU-5358336" "R-MMU-5358340" "R-MMU-5358343" "R-MMU-5362386"
[1857] "R-MMU-5362412" "R-MMU-5362427" "R-MMU-5362437" "R-MMU-5362441"
[1861] "R-MMU-5362459" "R-MMU-5362549" "R-MMU-5362551" "R-MMU-5362553"
[1865] "R-MMU-5632649" "R-MMU-5632652" "R-MMU-5632654" "R-MMU-9759844"
[1869] "R-MMU-9815762" "R-NUL-5362792" "R-MMU-109638"  "R-MMU-112381" 
[1873] "R-MMU-113402"  "R-MMU-113407"  "R-MMU-113409"  "R-MMU-6803523"
[1877] "R-MMU-72095"   "R-MMU-75095"   "R-MMU-77095"   "R-MMU-9012315"
[1881] "R-MMU-9012319" "R-MMU-9838035" "R-MMU-9838289" "R-MMU-8853734"
[1885] "R-MMU-8853737" "R-MMU-8853753" "R-MMU-8853755" "R-MMU-8853762"
[1889] "R-MMU-8853774" "R-MMU-8853792" "R-MMU-8853793" "R-MMU-8853801"
[1893] "R-MMU-8854897" "R-MMU-8854899" "R-MMU-8854905" "R-MMU-8854908"
[1897]

...

[4885] "R-MMU-508640"  "R-MMU-508741"  "R-MMU-549359"  "R-MMU-549451" 
[4889] "R-MMU-549498"  "R-MMU-5663146" "R-MMU-1678923" "R-MMU-1678944"
[4893] "R-MMU-71118"   "R-MMU-381412"  "R-MMU-381487"  "R-MMU-381503" 
[4897] "R-MMU-429749"  "R-MMU-429767"  "R-MMU-9620532" "R-MMU-9626966"
[4901] "R-MMU-383363"  "R-MMU-9033520" "R-MMU-9033995" "R-MMU-9034001"
[4905] "R-MMU-9034335" "R-MMU-9034351" "R-MMU-9034356" "R-NUL-9034341"
[4909] "R-MMU-71732"   "R-MMU-389491"  "R-MMU-428681"  "R-MMU-71146"  
[4913] "R-MMU-418848"  "R-MMU-418853"  "R-MMU-727749"  "R-MMU-727767" 
[4917] "R-MMU-9754929" "R-MMU-433698"  "R-MMU-5691131" "R-MMU-1810473"
[4921] "R-MMU-83582"   "R-MMU-9008684" "R-MMU-936941"  "R-MMU-9817397"
[4925] "R-MMU-9817411" "R-MMU-1306957" "R-MMU-1306963" "R-MMU-1306965"
[4929] "R-MMU-177923"  "R-MMU-177924"  "R-MMU-177925"  "R-MMU-177926" 
[4933] "R-MMU-177927"  "R-MMU-177930"  "R-MMU-177933"  "R-MMU-177934" 
[4937] "R-MMU-177935"  "R-MMU-177936"  "R-MMU-177937"  "R-MMU-177938" 
[4941]

[5345] "R-MMU-9668831" "R-NUL-9618916" "R-NUL-9619243" "R-MMU-418365" 
[5349] "R-MMU-265177"  "R-MMU-389026"  "R-MMU-209792"  "R-MMU-433104" 
[5353] "R-MMU-5340214" "R-MMU-5340226" "R-MMU-8982617" "R-MMU-418436" 
[5357] "R-MMU-3225851" "R-MMU-1483182" "R-MMU-200646"  "R-MMU-200729" 
[5361] "R-MMU-917870"  "R-MMU-444008"  "R-MMU-9714415" "R-MMU-2028651"
[5365] "R-MMU-3371527" "R-MMU-4793925" "R-MMU-8933452" "R-MMU-482619" 
[5369] "R-MMU-482621"  "R-MMU-6806877" "R-MMU-9748969" "R-MMU-9748999"
[5373] "R-MMU-9755013" "R-MMU-448954"  "R-MMU-448960"  "R-MMU-449198" 
[5377] "R-MMU-1454916" "R-MMU-194153"  "R-MMU-9750656" "R-MMU-9753283"
[5381] "R-MMU-8855711" "R-MMU-2066299" "R-MMU-4641147" "R-MMU-5138432"
[5385] "R-MMU-5138441" "R-MMU-508681"  "R-MMU-549505"  "R-MMU-5663147"
[5389] "R-MMU-5663263" "R-MMU-5663294" "R-MMU-5667072" "R-MMU-5667077"
[5393] "R-MMU-74885"   "R-MMU-8942208" "R-MMU-508749"  "R-MMU-549491" 
[5397] "R-MMU-5663158" "R-MMU-210444"  "R-MMU-264615"  "R-MMU-376369" 
[5401]

[5805] "R-MMU-70589"   "R-MMU-70600"   "R-MMU-8961021" "R-MMU-9624778"
[5809] "R-MMU-201443"  "R-MMU-201457"  "R-MMU-201475"  "R-MMU-202604" 
[5813] "R-MMU-419861"  "R-MMU-419841"  "R-MMU-6810472" "R-NUL-9011287"
[5817] "R-MMU-5082387" "R-MMU-158925"  "R-MMU-158942"  "R-MMU-158959" 
[5821] "R-MMU-158982"  "R-MMU-159001"  "R-MMU-159005"  "R-NUL-391979" 
[5825] "R-MMU-9759869" "R-MMU-9759884" "R-MMU-9759916" "R-MMU-9794025"
[5829] "R-MMU-9832924" "R-MMU-9832944" "R-MMU-5623508" "R-MMU-6803063"
[5833] "R-MMU-8948027" "R-MMU-389611"  "R-MMU-3076905" "R-MMU-4167509"
[5837] "R-MMU-5689649" "R-MMU-5690785" "R-MMU-5358494" "R-MMU-5649873"
[5841] "R-MMU-5651828" "R-MMU-422051"  "R-MMU-264622"  "R-MMU-9620456"
[5845] "R-MMU-9621179" "R-MMU-193666"  "R-MMU-205075"  "R-MMU-9008043"
[5849] "R-MMU-9692234" "R-MMU-9692260" "R-MMU-9733784" "R-MMU-9733795"
[5853] "R-MMU-9759850" "R-MMU-9795234" "R-MMU-9823863" "R-MMU-9823870"
[5857] "R-MMU-5661290" "R-MMU-111970"  "R-MMU-198314"  "R-MMU-212552" 
[5861]

[6265] "R-MMU-8987132" "R-MMU-8987202" "R-MMU-8987223" "R-MMU-909720" 
[6269] "R-MMU-912685"  "R-MMU-9678935" "R-MMU-9696179" "R-MMU-3299691"
[6273] "R-MMU-3299753" "R-MMU-3697860" "R-MMU-3777112" "R-MMU-3780959"
[6277] "R-MMU-3780963" "R-MMU-8951723" "R-MMU-5653878" "R-MMU-5653886"
[6281] "R-MMU-432065"  "R-MMU-75889"   "R-MMU-435349"  "R-MMU-425577" 
[6285] "R-MMU-8876696" "R-MMU-8858346" "R-MMU-9835358" "R-MMU-187828" 
[6289] "R-MMU-9624668" "R-MMU-444523"  "R-MMU-266070"  "R-MMU-3095901"
[6293] "R-MMU-9661405" "R-MMU-9843721" "R-MMU-5676917" "R-MMU-176474" 
[6297] "R-MMU-176669"  "R-MMU-212614"  "R-MMU-352119"  "R-NUL-9005747"
[6301] "R-NUL-9005752" "R-MMU-204392"  "R-MMU-204465"  "R-MMU-204485" 
[6305] "R-MMU-204600"  "R-MMU-373867"  "R-MMU-373875"  "R-MMU-375133" 
[6309] "R-MMU-375135"  "R-MMU-9645220" "R-MMU-449115"  "R-MMU-8983294"
[6313] "R-MMU-8983300" "R-MMU-8983307" "R-MMU-8983313" "R-MMU-8983335"
[6317] "R-MMU-8983356" "R-MMU-8983384" "R-MMU-9007126" "R-MMU-9007137"
[6321]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [12]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

In [13]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

In [14]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

  14.631010  14.802115  15.412860  14.744802  15.476610
ENSMUSG00000021772  12.838389  13.420172  13.649153  13.449860  13.461031
ENSMUSG00000021774  12.287199  12.180964  13.040892  12.175635  12.493601
ENSMUSG00000021775  11.262691  11.449542  16.364303  13.731838  16.323335
ENSMUSG00000021779  12.402088  13.295830  12.749594  13.448202  13.479613
ENSMUSG00000021782   9.964444   8.793114   8.863296   8.963041   8.866146
ENSMUSG00000021785  12.779547  14.028271  13.362617  13.218437  13.694034
ENSMUSG00000021786  13.257208  13.569259  13.369566  12.353552  13.400323
ENSMUSG00000021789   6.454101   6.957527   7.439565   8.469470   8.634615
ENSMUSG00000021794  19.182957  19.914730  19.485457  19.586482  19.645909
ENSMUSG00000021795   7.856004   7.212420   7.460513   7.794721   7.420313
ENSMUSG00000021796  12.672957  13.005066  12.935217  13.354069  12.840354
ENSMUSG00000021799   4.537701   4.691374   5.869749   4.674013   4.674317
ENSMUSG00000021804   7.438853   7.580261   7.812808   6.

ENSMUSG00000022148  10.594703  11.717371  11.524627  12.661616  11.063135
ENSMUSG00000022149  16.292009  18.889779  17.930689  18.311171  18.520830
ENSMUSG00000022150  11.612953  10.445141   9.749423  10.584413   9.991618
ENSMUSG00000022159   8.679462   8.412491   8.675337   9.024664   8.404420
ENSMUSG00000022160  11.444604  11.676437  11.666892  11.783135  11.151373
ENSMUSG00000022175  14.920647  13.883507  14.207360  14.141382  13.993720
ENSMUSG00000022177  10.620971  10.504565  11.405993   9.630528  11.555877
ENSMUSG00000022178   9.419856  11.379961   9.572070  10.626628   9.509303
ENSMUSG00000022180  10.097025   7.524667   7.518759   9.028760   7.922699
ENSMUSG00000022181  11.431993  17.779820  15.313153  16.236168  16.727315
ENSMUSG00000022184   9.634882  10.015807  10.923816  10.407491  10.291873
ENSMUSG00000022185  15.195503  14.724331  14.409993  14.309515  14.626041
ENSMUSG00000022186   8.494514   8.107487   8.743055   9.156319   7.559706
ENSMUSG00000022193  14.488777  14.9349

ENSMUSG00000022477  16.658835  17.005338  17.341284  16.880986  17.550726
ENSMUSG00000022479   5.781711   4.691374   4.760844   6.150928   4.674317
ENSMUSG00000022483   4.537701   4.691374   4.760844   4.674013   4.674317
ENSMUSG00000022488  10.496422   8.489151   8.276869  11.811094   8.235652
ENSMUSG00000022489   7.903438   6.808423   7.661512   9.973218   7.574842
ENSMUSG00000022496   4.537701   4.691374   4.760844   4.674013   5.686372
ENSMUSG00000022501   5.398812   4.691374   4.760844   4.674013   4.674317
ENSMUSG00000022510   4.537701   4.691374   4.760844   6.150928   4.674317
ENSMUSG00000022512  15.781948  16.112344  12.347052  14.767793  12.521385
ENSMUSG00000022514  16.388543  17.291575  17.385122  16.860388  17.436424
ENSMUSG00000022517  14.801089  14.838849  15.366174  14.958672  15.338519
ENSMUSG00000022525   4.537701   4.691374   4.760844   4.674013   4.674317
ENSMUSG00000022528  10.937684  10.494573   9.261378  10.367507   9.118347
ENSMUSG00000022529  11.719863  11.4402

ENSMUSG00000022838   6.446247   6.444604   6.399896   6.722390   6.795578
ENSMUSG00000022840   8.074260   8.248113   6.399896   7.474350   7.522263
ENSMUSG00000022841  13.308245  13.754270  13.918699  13.654279  14.002875
ENSMUSG00000022842   4.537701   4.691374   4.760844   4.674013   4.674317
ENSMUSG00000022843  13.809814  14.679904  14.934195  14.231486  15.153755
ENSMUSG00000022853  19.000498  18.529903  18.257486  17.120833  18.330972
ENSMUSG00000022855  12.155296  12.437355  12.603018  12.860590  12.358180
ENSMUSG00000022858  11.578737  12.462820  11.003075  12.318881  11.520691
ENSMUSG00000022861   5.615858   4.691374   4.760844   6.298996   4.674317
ENSMUSG00000022865  14.093651  16.262202  15.077368  15.967112  15.273945
ENSMUSG00000022867  12.072047  13.121712  13.124069  12.689793  13.457919
ENSMUSG00000022868  22.280149  21.931429  22.103562  21.958592  22.142636
ENSMUSG00000022877  19.741856  19.316505  19.899981  19.872866  19.727852
ENSMUSG00000022878   4.537701   4.6913

ENSMUSG00000023274   7.602854   8.157362   7.581109   7.380112   7.377514
ENSMUSG00000023277   8.545818   7.788241   7.380135   8.437235   6.877043
ENSMUSG00000023279   4.537701   4.691374   4.760844   5.685735   4.674317
ENSMUSG00000023284   7.743994   8.442030   7.741366   8.037417   8.212076
ENSMUSG00000023286  11.521222  11.874215  11.970388  11.480548  12.007719
ENSMUSG00000023341   9.803975   8.724215   9.160285   9.349909   8.821269
ENSMUSG00000023349   8.297161   6.876964   6.399896   8.903252   7.187639
ENSMUSG00000023439   4.537701   4.691374   4.760844   4.674013   4.674317
ENSMUSG00000023456  17.893082  18.248794  18.069344  17.758994  18.092565
ENSMUSG00000023460  12.298451  12.218551  12.398785  12.015661  12.151079
ENSMUSG00000023723  11.893720  11.527644  11.515631  11.737735  12.079702
ENSMUSG00000023764  11.356668  12.126167  12.245448  12.590305  13.226863
ENSMUSG00000023781   5.630261   6.352232   6.183202   6.150928   5.686372
ENSMUSG00000023791   9.896912  10.2329

...

ENSMUSG00000020241  11.966899  10.991779   9.997709  10.269057   9.870114
ENSMUSG00000020246  11.167619  11.305773  11.634669  11.694853  10.901962
ENSMUSG00000020248  11.065352  11.158687  10.575500  11.079157  12.746508
ENSMUSG00000020250  17.651396  17.995555  18.049258  18.023820  18.455649
ENSMUSG00000020256   7.200768   7.691618   7.370525   7.103420   6.118985
ENSMUSG00000020257  13.153162  13.381564  12.996114  12.938682  13.471149
ENSMUSG00000020258  11.930039  10.458601  11.384455  11.144209  14.215994
ENSMUSG00000020260  12.381780  12.228899  12.111022  12.178278  11.375025
ENSMUSG00000020261  11.297213  11.319873  10.779374  11.225655  11.346347
ENSMUSG00000020262  10.095867  10.414727  10.362351   9.401712   9.444949
ENSMUSG00000020264   7.735777   7.172187   6.319151   6.851516   5.661706
ENSMUSG00000020265  14.497067  13.701966  14.093862  14.110506  14.487536
ENSMUSG00000020271  13.992613  14.289181  14.299455  14.139886  13.558585
ENSMUSG00000020272  10.321234  11.1114

ENSMUSG00000020534  11.408435  11.240826  11.881138  11.534543  13.633662
ENSMUSG00000020537  13.137871  13.537136  13.597988  13.613079  13.379298
ENSMUSG00000020538  12.957331  12.546032  12.762885  13.001953  14.793114
ENSMUSG00000020542   5.946361   5.508396   4.586217   4.621512   4.662507
ENSMUSG00000020544  10.595824  10.631525  10.912746  10.874152  10.279869
ENSMUSG00000020553  14.210694  13.622564  13.638046  13.103319  13.469611
ENSMUSG00000020561  11.220256  11.272061  10.706108  11.073510  11.107060
ENSMUSG00000020566   6.117676   6.178374   5.673335   6.458813   5.820422
ENSMUSG00000020571  16.723472  17.014751  16.724639  17.972033  16.087955
ENSMUSG00000020572  15.277359  15.830506  15.972359  15.032271  13.843641
ENSMUSG00000020573   7.881087   8.543303   8.013236   8.058370   7.467028
ENSMUSG00000020576  11.511349  12.143559  12.190076  12.160046  12.017542
ENSMUSG00000020580  14.490875  14.791216  14.662964  14.628250  13.551360
ENSMUSG00000020583   4.599237   4.5882

ENSMUSG00000020850  15.720292  16.336843  16.094683  16.083395  16.801131
ENSMUSG00000020857  14.861440  14.282473  14.567180
ENSMUSG00000019971   9.139726
ENSMUSG00000019977  14.113783
ENSMUSG00000019978  11.390980
ENSMUSG00000019979   9.892884
ENSMUSG00000019982   5.825812
ENSMUSG00000019984  10.411675
ENSMUSG00000019986   7.621911
ENSMUSG00000019987  19.877656
ENSMUSG00000019988  11.749812
ENSMUSG00000019990   8.845196
ENSMUSG00000020003  13.069268
ENSMUSG00000020007   4.621722
ENSMUSG00000020009  12.401142
ENSMUSG00000020017  14.977297
ENSMUSG00000020018   8.257834
ENSMUSG00000020019  10.158221
ENSMUSG00000020020   4.621722
ENSMUSG00000020022  11.513861
ENSMUSG00000020024  10.898983
ENSMUSG00000020027  10.520789
ENSMUSG00000020029  16.378719
ENSMUSG00000020032   8.202255
ENSMUSG00000020038   8.646633
ENSMUSG00000020044  13.929836
ENSMUSG00000020048  18.968096
ENSMUSG00000020051  17.334617
ENSMUSG00000020053  17.169677
ENSMUSG00000020061   5.577106
ENSMUSG00000020062   5.577106
ENSM

ENSMUSG00000022422   6.137539
ENSMUSG00000022426  15.300156
ENSMUSG00000022427  12.975932
ENSMUSG00000022428   9.200505
ENSMUSG00000022429   5.833966
ENSMUSG00000022432   6.823616
ENSMUSG00000022433  10.700145
ENSMUSG00000022436   8.377452
ENSMUSG00000022437  14.906859
ENSMUSG00000022438   8.686242
ENSMUSG00000022442   7.863873
ENSMUSG00000022443  16.154609
ENSMUSG00000022449   5.577106
ENSMUSG00000022450  14.669978
ENSMUSG00000022451  14.180583
ENSMUSG00000022452  13.707205
ENSMUSG00000022454   4.621722
ENSMUSG00000022462  16.222142
ENSMUSG00000022463  13.585750
ENSMUSG00000022464  18.759489
ENSMUSG00000022469   8.165887
ENSMUSG00000022471  10.963510
ENSMUSG00000022474  13.119927
ENSMUSG00000022475  10.465692
ENSMUSG00000022476  11.635213
ENSMUSG00000022477  15.209941
ENSMUSG00000022479   6.253582
ENSMUSG00000022483   4.621722
ENSMUSG00000022488  10.897778
ENSMUSG00000022489   8.423667
ENSMUSG00000022496   4.621722
ENSMUSG00000022501   4.621722
ENSMUSG00000022510   5.833966
ENSMUSG000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

      Resample1
 [1,]         1
 [2,]         2
 [3,]         3
 [4,]         4
 [5,]         5
 [6,]         6
 [7,]         7
 [8,]         8
 [9,]         9
[10,]        10
[11,]        11
[12,]        12
[13,]        13
[14,]        14
[15,]        15
[16,]        16
[17,]        17
[18,]        18
[19,]        19
[20,]        20
[21,]        21
[22,]        22
[23,]        23
[24,]        24
[25,]        25
[26,]        26


In [16]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

                   SRR3593523 SRR7817700 SRR7817648 SRR7817672 SRR7817662
ENSMUSG00000000001  14.547886  15.798199  15.281283  15.275252  15.736033
ENSMUSG00000000028   6.923071   7.650167   6.913712   7.198065   7.191227
ENSMUSG00000000049  20.789011  20.246032  20.562958  20.083925  20.463679
ENSMUSG00000000058   8.153123   7.727127   7.585587   7.994352   7.792455
ENSMUSG00000000085  10.513756   9.724836  10.184706  10.061571   9.490532
ENSMUSG00000000088  15.092601  16.822697  17.012867  15.960130  16.829317
ENSMUSG00000000120   9.178748   8.520550   7.733114   7.213414   6.723577
ENSMUSG00000000126   7.201449   6.447109   6.573393   6.722390   7.213750
ENSMUSG00000000127   9.102349   9.447231   9.126580   9.181653   9.231817
ENSMUSG00000000142  11.811011   9.936467   9.358383   9.431700   8.691914
ENSMUSG00000000149  15.676300  15.833942  16.915542  15.834348  16.250161
ENSMUSG00000000154  15.865069  15.498889  15.818053  15.024826  15.809272
ENSMUSG00000000168  13.769196  14.7780

ENSMUSG00000001105  12.897176  13.142184  12.749594  13.389744  13.095021
ENSMUSG00000001119  10.805716   9.073938   8.148232   9.639700   8.444120
ENSMUSG00000001123  19.109639  18.360580  18.067256  17.191367  17.901841
ENSMUSG00000001127  15.377449  15.139629  15.317995  14.930353  15.213624
ENSMUSG00000001128  13.400591  10.051705   9.873761  13.798655  11.018855
ENSMUSG00000001131   6.284377   6.006169   4.760844   7.085748   4.674317
ENSMUSG00000001143  11.871885  11.376183  10.280859  11.410513  10.921921
ENSMUSG00000001155  18.554853  18.960325  19.088849  18.224637  19.225116
ENSMUSG00000001158  10.885067  10.519319  10.145019  10.272039  10.348843
ENSMUSG00000001173  10.064029   9.276347   9.393148   9.746085   9.347493
ENSMUSG00000001175  16.277045  17.713032  17.213285  17.479234  17.156984
ENSMUSG00000001211  17.361884  17.138774  17.307399  16.731872  17.358584
ENSMUSG00000001225   4.537701   4.691374   4.760844   4.674013   5.542842
ENSMUSG00000001228   6.778327   6.4814

ENSMUSG00000002249  11.385803  11.036023  10.896166  10.421984  10.660213
ENSMUSG00000002250  11.434131  13.934782  10.037902  10.382525   9.578150
ENSMUSG00000002257   6.534645   6.335364   5.994580   6.794345   6.079196
ENSMUSG00000002279  13.595044  13.580516  13.450343  13.256038  13.928746
ENSMUSG00000002289  16.633725  15.161121  14.663689  13.609085  16.318281
ENSMUSG00000002297   7.468591   8.548222   7.304390   7.709118   8.582698
ENSMUSG00000002307  11.758870  10.932278  10.859019  11.240833  10.499321
ENSMUSG00000002308   8.504479   9.107025   8.937566   8.971927   9.033675
ENSMUSG00000002324   8.183530   7.639198   7.737244   7.009067   7.423402
ENSMUSG00000002325  14.383111  13.085525  12.694798  13.373141  13.194517
ENSMUSG00000002326  10.967774  11.288840  11.148400  10.376331  10.941042
ENSMUSG00000002346  17.098611  15.910229  15.968849  15.540974  16.130191
ENSMUSG00000002365  13.555195  13.822037  14.273848  13.501992  14.199545
ENSMUSG00000002379  13.959659  14.0741

ENSMUSG00000003379   7.788597   4.691374   6.183202   6.150928   6.151777
ENSMUSG00000003402  16.503922  15.885738  15.974723  15.917837  16.075875
ENSMUSG00000003411   6.341810   7.227875   6.360896   7.374832   6.151777
ENSMUSG00000003418   5.781711   4.691374   4.760844   4.674013   4.674317
ENSMUSG00000003421  12.088249  12.237680  12.632894  12.007556  12.090048
ENSMUSG00000003429  15.283102  14.067041  13.295567  12.803915  13.965327
ENSMUSG00000003435  14.975439  14.988113  14.626546  15.340646  14.623236
ENSMUSG00000003436   4.537701   4.691374   4.760844   4.674013   4.674317
ENSMUSG00000003437  12.957398  12.729624  13.266001  12.612433  13.132895
ENSMUSG00000003444   8.816619   9.571650   8.502103   8.756660   8.414081
ENSMUSG00000003452   5.630261   6.006169   4.760844   5.962843   5.963600
ENSMUSG00000003458  14.438543  13.512601  14.193457  13.934665  14.075302
ENSMUSG00000003464  16.489207  15.890784  16.089243  15.526962  16.305918
ENSMUSG00000003476   6.876085   6.1961

ENSMUSG00000004654   5.630261   4.691374   5.869749   4.674013   4.674317
ENSMUSG00000004655  15.657265  14.098538  13.306553  13.728240  13.950213
ENSMUSG00000004665   9.097804   8.072021   7.737244   9.552720   8.103071
ENSMUSG00000004667  13.060079  12.591433  12.812659  12.220492  12.588181
ENSMUSG00000004668   4.537701   4.691374   4.760844   6.414595   4.674317
ENSMUSG00000004677  12.963092  12.932513  13.500604  13.832888  13.040103
ENSMUSG00000004709   6.080424   4.691374   4.760844   6.805639   5.963600
ENSMUSG00000004730  11.318613   9.483166   8.047050  12.852833   9.020859
ENSMUSG00000004768   9.000434   9.139194  10.266850   9.548434  10.185706
ENSMUSG00000004771  13.440039  14.295398  14.549892  14.091960  14.410951
ENSMUSG00000004788  11.529487  12.021212  12.180710  11.655486  11.352597
ENSMUSG00000004789  16.916223  17.449565  17.179176  16.506189  17.358089
ENSMUSG00000004791   4.537701   4.691374   4.760844   4.674013   4.674317
ENSMUSG00000004815  13.408193  13.8662

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




                   SRR3593580
ENSMUSG00000000001  15.614499
ENSMUSG00000000028   7.123074
ENSMUSG00000000049  19.056622
ENSMUSG00000000058   8.974806
ENSMUSG00000000085  10.274974
ENSMUSG00000000088  14.841971
ENSMUSG00000000120   7.479000
ENSMUSG00000000126   5.957096
ENSMUSG00000000127   8.566952
ENSMUSG00000000142  12.051242
ENSMUSG00000000149  14.840522
ENSMUSG00000000154  14.195524
ENSMUSG00000000168  12.995104
ENSMUSG00000000171  16.068702
ENSMUSG00000000182   6.024992
ENSMUSG00000000183   4.621722
ENSMUSG00000000184  11.327384
ENSMUSG00000000197   5.577106
ENSMUSG00000000214   4.621722
ENSMUSG00000000215   4.621722
ENSMUSG00000000216   4.621722
ENSMUSG00000000223   5.577106
ENSMUSG00000000247   7.006458
ENSMUSG00000000253   8.494457
ENSMUSG00000000263   4.621722
ENSMUSG00000000275  15.394553
ENSMUSG00000000276  10.071897
ENSMUSG00000000290  10.938471
ENSMUSG00000000296   8.972881
ENSMUSG00000000301  16.273960
ENSMUSG00000000303  15.873642
ENSMUSG00000000305   7.682498
ENSMUSG00

ENSMUSG00000002814   9.684628
ENSMUSG00000002820  11.768707
ENSMUSG00000002846  11.409071
ENSMUSG00000002847   9.741963
ENSMUSG00000002870   9.309842
ENSMUSG00000002885  12.203254
ENSMUSG00000002900  10.048039
ENSMUSG00000002908   7.359673
ENSMUSG00000002944  19.101938
ENSMUSG00000002948  12.915048
ENSMUSG00000002957  13.706629
ENSMUSG00000002963  11.400710
ENSMUSG00000002968  14.158412
ENSMUSG00000002983   8.046214
ENSMUSG00000002984  12.068082
ENSMUSG00000002985  21.877429
ENSMUSG00000002992  15.278636
ENSMUSG00000003031  14.817157
ENSMUSG00000003032   8.278421
ENSMUSG00000003033  12.351689
ENSMUSG00000003037  13.959068
ENSMUSG00000003039  14.033974
ENSMUSG00000003051   8.488478
ENSMUSG00000003053  19.363317
ENSMUSG00000003062   9.920796
ENSMUSG00000003068  14.939266
ENSMUSG00000003070   5.577106
ENSMUSG00000003072  16.023633
ENSMUSG00000003099  12.147503
ENSMUSG00000003119  12.431352
ENSMUSG00000003131  13.378866
ENSMUSG00000003135  11.521879
ENSMUSG00000003153   7.870444
ENSMUSG000

ENSMUSG00000005683  15.689157
ENSMUSG00000005686   8.455576
ENSMUSG00000005687  12.525087
ENSMUSG00000005698  13.044153
ENSMUSG00000005699   7.377707
ENSMUSG00000005705   5.453062
ENSMUSG00000005716   4.621722
ENSMUSG00000005763   9.044401
ENSMUSG00000005779  15.630353
ENSMUSG00000005800  12.362816
ENSMUSG00000005803  14.978754
ENSMUSG00000005804  10.657821
ENSMUSG00000005813  13.227351
ENSMUSG00000005824   8.377452
ENSMUSG00000005836   9.462572
ENSMUSG00000005864   4.621722
ENSMUSG00000005871  14.365344
ENSMUSG00000005883   5.577106
ENSMUSG00000005886  13.282056
ENSMUSG00000005892   4.621722
ENSMUSG00000005893  13.433312
ENSMUSG00000005897  11.571376
ENSMUSG00000005899  11.315314
ENSMUSG00000005907  12.769363
ENSMUSG00000005947   7.173881
ENSMUSG00000005949  12.556889
ENSMUSG00000005950   6.255687
ENSMUSG00000005951  13.715627
ENSMUSG00000005952   6.251471
ENSMUSG00000005958   6.142459
ENSMUSG00000005973  10.860969
ENSMUSG00000005994   4.621722
ENSMUSG00000006005  15.621342
ENSMUSG000

ENSMUSG00000012282   4.621722
ENSMUSG00000012396   5.833966
ENSMUSG00000012405  14.387174
ENSMUSG00000012443   8.247729
ENSMUSG00000012483   8.034623
ENSMUSG00000012519  11.435487
ENSMUSG00000012609   9.030466
ENSMUSG00000012705   4.621722
ENSMUSG00000012819   5.577106
ENSMUSG00000012848  16.554525
ENSMUSG00000013076  10.297090
ENSMUSG00000013160  14.687299
ENSMUSG00000013236   9.280549
ENSMUSG00000013275   9.868144
ENSMUSG00000013418   6.617909
ENSMUSG00000013465  11.988131
ENSMUSG00000013584   6.794056
ENSMUSG00000013593  16.866986
ENSMUSG00000013629  10.352756
ENSMUSG00000013643   9.160007
ENSMUSG00000013662  14.836727
ENSMUSG00000013663  15.404720
ENSMUSG00000013698  11.539753
ENSMUSG00000013707   7.979733
ENSMUSG00000013787  14.644002
ENSMUSG00000013833  10.723745
ENSMUSG00000013846  16.695993
ENSMUSG00000013921   6.791658
ENSMUSG00000013936   4.621722
ENSMUSG00000013974   7.350976
ENSMUSG00000014030   8.336743
ENSMUSG00000014074  11.678599
ENSMUSG00000014075   7.954643
ENSMUSG000

In [17]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

In [18]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030_time_cross_20.Rds",sep=""))

In [19]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

In [20]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

In [21]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [22]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030_time_cross_20.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 reactions (19.3%)..."
[1] "Processed 2000 of 9825 reactions (20.4%)..."
[1] "Storing PCA objects containing reactions 1000-2000 of 9825 reactions (20.4%)..."
[1] "Processed 2100 of 9825 reactions (21.4%)..."
[1] "Processed 2200 of 9825 reactions (22.4%)..."
[1] "Processed 2300 of 9825 reactions (23.4%)..."
[1] "Processed 2400 of 9825 reactions (24.4%)..."
[1] "Processed 2500 of 9825 reactions (25.5%)..."
[1] "Processed 2600 of 9825 reactions (26.5%)..."
[1] "Processed 2700 of 9825 reactions (27.5%)..."
[1] "Processed 2800 of 9825 reactions (28.5%)..."
[1] "Processed

In [23]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030_time_cross_20.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030_time_cross_20.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6951267 371.3   13076164 698.4  9551737 510.2
Vcells 13461063 102.7   25842636 197.2 25841572 197.2


In [24]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030_time_cross_20.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030_time_cross_20.Rds", sep = ""))

In [25]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

[1]  1  9 14
[1] "----------"
[1] 10 21
[1] "----------"
[1]  5 17
[1] "----------"
[1] 12 15 24
[1] "----------"
[1]  7 19 25
[1] "----------"
[1]  4  8 26
[1] "----------"
[1] 11 22
[1] "----------"
[1] 13 16 18
[1] "----------"
[1]  3 20 23
[1] "----------"
[1] 2 6
[1] "----------"


In [ ]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
       
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  NaN : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  NaN : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  NaN : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  NaN : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-3247837 : Last ARI =  NaN : Last ECOUNT =  19 : Last Lung MISCLASS 

[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  NaN : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8936481 : Last ARI =  NaN : Last ECOUNT =  77 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9677051 : Last ARI =  NaN : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5336403 : Last ARI =  NaN : Last ECOUNT =  72 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-72671 : Last ARI =  NaN : Last ECOUNT =  90 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94 % remaining..."
[1] "Last RXN_ID =  R-MMU-9633741 : Last ARI =  NaN : Last ECOUNT =  85 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  93.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-110371 : Last ARI =  NaN : Last ECOUNT =  18 : Last Lung MISCLASS =

[1] "Last RXN_ID =  R-MMU-200652 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9624798 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-2213248 : Last ARI =  NaN : Last ECOUNT =  64 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5618080 : Last ARI =  NaN : Last ECOUNT =  40 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8852298 : Last ARI =  NaN : Last ECOUNT =  17 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9648114 : Last ARI =  NaN : Last ECOUNT =  108 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-114670 : Last ARI =  NaN : Last ECOUNT =  7 : Last Lung MISCLASS

[1] "Last RXN_ID =  R-MMU-264458 : Last ARI =  NaN : Last ECOUNT =  50 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-5635854 : Last ARI =  NaN : Last ECOUNT =  52 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-68948 : Last ARI =  NaN : Last ECOUNT =  50 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8939801 : Last ARI =  NaN : Last ECOUNT =  47 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-8854628 : Last ARI =  NaN : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83 % remaining..."
[1] "Last RXN_ID =  R-MMU-5696664 : Last ARI =  NaN : Last ECOUNT =  16 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  82.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9634443 : Last ARI =  NaN : Last ECOUNT =  3 : Last Lung MISCLASS = 

[1] "Last RXN_ID =  R-MMU-8984012 : Last ARI =  NaN : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  77.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5666160 : Last ARI =  NaN : Last ECOUNT =  94 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  77.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-201685 : Last ARI =  NaN : Last ECOUNT =  26 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  77.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-72722 : Last ARI =  NaN : Last ECOUNT =  8 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  77.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9674816 : Last ARI =  NaN : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  77.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9682572 : Last ARI =  NaN : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  77.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-9817994 : Last ARI =  NaN : Last ECOUNT =  7 : Last Lung MISCLASS = 

...

[1] "Last RXN_ID =  R-MMU-198275 : Last ARI =  NaN : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  23 % remaining..."
[1] "Last RXN_ID =  R-MMU-114683 : Last ARI =  NaN : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-75090 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-77342 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-449937 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9718020 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6806966 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last 

[1] "Last RXN_ID =  R-MMU-9668464 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2454118 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-159425 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-6785607 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-6807224 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17 % remaining..."
[1] "Last RXN_ID =  R-MMU-210292 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  16.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-1676024 : Last ARI =  NaN : Last ECOUNT =  3 : Last Lung MISCLASS =   : 

[1] "Last RXN_ID =  R-MMU-8848873 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12 % remaining..."
[1] "Last RXN_ID =  R-MMU-6792445 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  11.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5692764 : Last ARI =  NaN : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  11.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-2408515 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  11.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9753027 : Last ARI =  NaN : Last ECOUNT =  20 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  11.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-1855182 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  11.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9626914 : Last ARI =  NaN : Last ECOUNT =  3 : Last Lung MISCLASS =  

[1] "Last RXN_ID =  R-MMU-5603114 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  6.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2730862 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  6.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-389919 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  6.3 % remaining..."
[1] "Last RXN_ID =  R-NUL-1248744 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  6.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-8857692 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  6.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-6805479 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  6 % remaining..."
[1] "Last RXN_ID =  R-MMU-187246 : Last ARI =  NaN : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last U

[1] "Last RXN_ID =  R-MMU-75823 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  1 % remaining..."
[1] "Last RXN_ID =  R-MMU-977389 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  0.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-981713 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  0.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-188979 : Last ARI =  NaN : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  0.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6786245 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  0.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-5696004 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  0.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-5619427 : Last ARI =  NaN : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uter

In [28]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030_time_cross_20.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030_time_cross_20.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030_time_cross_20.Rds", sep = ""))

In [29]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030_time_cross_20.Rds", sep = ""))

                           RXN_ID MISCLASS           ARI ECOUNT
R-MMU-170666.0.00    R-MMU-170666        0           NaN     14
R-MMU-170666.30.00   R-MMU-170671        0           NaN     14
R-MMU-170671.0.00    R-MMU-170674        0           NaN     14
R-MMU-170671.30.00   R-MMU-170686        0           NaN     14
R-MMU-170674.0.00    R-MMU-380073        0           NaN    182
R-MMU-170674.30.00   R-MMU-392187        0           NaN     22
R-MMU-170686.0.00    R-MMU-392195        0           NaN     22
R-MMU-170686.30.00   R-MMU-392202        0           NaN     22
R-MMU-380073.0.00    R-MMU-749454        0           NaN    178
R-MMU-380073.30.00   R-MMU-749456        0           NaN    178
R-MMU-392187.0.00    R-MMU-751001        0           NaN     21
R-MMU-392187.30.00  R-MMU-8964229        0           NaN      5
R-MMU-392195.0.00   R-MMU-8964242        0           NaN      5
R-MMU-392195.30.00  R-MMU-8964329        0           NaN      5
R-MMU-392202.0.00   R-MMU-8982012       

R-MMU-205099.30.00  R-MMU-5654224        0           NaN     12
R-MMU-205112.0.00   R-MMU-5654392        0           NaN     20
R-MMU-205112.30.00  R-MMU-5654397        0           NaN     16
R-MMU-205117.0.00   R-MMU-5654399        0           NaN     16
R-MMU-205117.30.00  R-MMU-5654402        0           NaN     20
R-MMU-209532.0.00   R-MMU-5654404        0           NaN     16
R-MMU-209532.30.00  R-MMU-5654406        0           NaN     18
R-MMU-209566.0.00   R-MMU-5654407        0           NaN     16
R-MMU-209566.30.00  R-MMU-5654408        0           NaN     12
R-MMU-9012761.0.00  R-MMU-5654409        0           NaN     12
R-MMU-9012761.30.00 R-MMU-5654413        0           NaN     17
R-MMU-3247837.0.00  R-MMU-5654416        0           NaN     15
R-MMU-3247837.30.00 R-MMU-5654418        0           NaN     14
R-MMU-3247840.0.00  R-MMU-5654422        0           NaN     14
R-MMU-3247840.30.00 R-MMU-5654423        0           NaN     16
R-MMU-3247843.0.00  R-MMU-5654425       

R-MMU-5654224.30.00 R-MMU-9011082        0           NaN      2
R-MMU-5654392.0.00  R-MMU-9011144        0           NaN      3
R-MMU-5654392.30.00 R-MMU-9011146        0           NaN      3
R-MMU-5654397.0.00  R-MMU-9753028        0           NaN      5
R-MMU-5654397.30.00 R-MMU-9753126        0           NaN      5
R-MMU-5654399.0.00   R-MMU-514604        0           NaN      2
R-MMU-5654399.30.00  R-MMU-975389        0           NaN      4
R-MMU-5654402.0.00  R-MMU-1169394        0           NaN      5
R-MMU-5654402.30.00 R-MMU-1169398        0           NaN      5
R-MMU-5654404.0.00  R-MMU-1169402        0           NaN      6
R-MMU-5654404.30.00 R-MMU-1169403        0           NaN      4
R-MMU-5654406.0.00  R-MMU-1169405        0           NaN      5
R-MMU-5654406.30.00 R-MMU-1169406        0           NaN     12
R-MMU-5654407.0.00  R-MMU-1678842        0           NaN     13
R-MMU-5654407.30.00 R-MMU-1678843        0           NaN      5
R-MMU-5654408.0.00  R-MMU-5653754       

R-MMU-5654659.30.00 R-MMU-2424487        0           NaN     22
R-MMU-5654662.0.00   R-MMU-388814        0           NaN      6
R-MMU-5654662.30.00  R-MMU-388829        0           NaN      9
R-MMU-5654663.0.00   R-MMU-388831        0           NaN      8
R-MMU-5654663.30.00  R-MMU-388832        0           NaN      9
R-MMU-5654664.0.00   R-MMU-388833        0           NaN     17
R-MMU-5654664.30.00  R-MMU-389158        0           NaN      9
R-MMU-5654667.0.00   R-MMU-389348        0           NaN      8
R-MMU-5654667.30.00  R-MMU-389350        0           NaN      8
R-MMU-5654669.0.00   R-MMU-389352        0           NaN      7
R-MMU-5654669.30.00  R-MMU-389354        0           NaN      7
R-MMU-5654672.0.00   R-MMU-389381        0           NaN      6
R-MMU-5654672.30.00  R-MMU-389758        0           NaN     22
R-MMU-5654673.0.00   R-MMU-389759        0           NaN      7
R-MMU-5654673.30.00  R-MMU-389762        0           NaN      5
R-MMU-5654677.0.00   R-MMU-434836       

R-MMU-9011082.30.00 R-MMU-9692139        0           NaN      2
R-MMU-9011144.0.00  R-MMU-9692145        0           NaN      2
R-MMU-9011144.30.00 R-MMU-9692156        0           NaN      4
R-MMU-9011146.0.00  R-MMU-9692157        0           NaN      4
R-MMU-9011146.30.00 R-MMU-9692268        0           NaN      2
R-MMU-9753028.0.00   R-MMU-450434        0           NaN      1
R-MMU-9753028.30.00  R-MMU-450466        0           NaN     45
R-MMU-9753126.0.00   R-MMU-450551        0           NaN      7
R-MMU-9753126.30.00  R-MMU-450580        0           NaN     12
R-MMU-514604.0.00     R-MMU-72103        0           NaN     43
R-MMU-514604.30.00  R-MMU-9770119        0           NaN     69
R-MMU-975389.0.00   R-MMU-9770141        0           NaN    123
R-MMU-975389.30.00  R-MMU-9770142        0           NaN    132
R-MMU-1169394.0.00  R-MMU-9770145        0           NaN    145
R-MMU-1169394.30.00 R-MMU-9770236        0           NaN    119
R-MMU-1169398.0.00  R-MMU-6798748       

...

R-MMU-1483219.0.00  R-MMU-8953111        0           NaN      2
R-MMU-1483219.30.00  R-MMU-199626        0           NaN      2
R-MMU-1483229.0.00  R-MMU-9705794        0           NaN      1
R-MMU-1483229.30.00  R-MMU-374673        0           NaN      1
R-MMU-5693691.0.00   R-MMU-209765        0           NaN      1
R-MMU-5693691.30.00  R-MMU-211950        0           NaN      1
R-MMU-9749647.0.00   R-MMU-192033        0           NaN      1
R-MMU-9749647.30.00  R-MMU-192067        0           NaN      1
R-MMU-9749792.0.00   R-MMU-193746        0           NaN      1
R-MMU-9749792.30.00  R-MMU-193755        0           NaN      1
R-MMU-5696119.0.00   R-MMU-193821        0           NaN      1
R-MMU-5696119.30.00  R-MMU-193824        0           NaN      1
R-MMU-9673827.0.00  R-NUL-1236960        0           NaN      1
R-MMU-9673827.30.00 R-MMU-8874239        0           NaN      2
R-MMU-2872444.0.00  R-MMU-8874322        0           NaN      1
R-MMU-2872444.30.00 R-MMU-8874406       

R-MMU-4551446.0.00   R-MMU-427570        0           NaN      2
R-MMU-4551446.30.00 R-MMU-2161526        0           NaN      1
R-MMU-6807585.0.00  R-MMU-9659680        0           NaN      1
R-MMU-6807585.30.00 R-MMU-9759454        0           NaN      1
R-MMU-390912.0.00   R-MMU-9759461        0           NaN      1
R-MMU-390912.30.00   R-MMU-372819        0           NaN      1
R-MMU-9692539.0.00  R-MMU-5606210        0           NaN      3
R-MMU-9692539.30.00 R-MMU-5606256        0           NaN      3
R-MMU-202709.0.00   R-MMU-5629230        0           NaN      4
R-MMU-202709.30.00  R-MMU-8942575        0           NaN      2
R-MMU-1254248.0.00  R-MMU-8953339        0           NaN      1
R-MMU-1254248.30.00 R-MMU-5696839        0           NaN      1
R-MMU-1254251.0.00  R-MMU-6801456        0           NaN      1
R-MMU-1254251.30.00 R-MMU-8944219        0           NaN      1
R-MMU-2032770.0.00   R-MMU-879594        0           NaN      1
R-MMU-2032770.30.00  R-MMU-211874       

R-MMU-9036987.0.00   R-MMU-187246        0           NaN      2
R-MMU-9036987.30.00  R-MMU-189119        0           NaN      2
R-MMU-9603982.0.00   R-MMU-391942        0           NaN      1
R-MMU-9603982.30.00 R-MMU-9726949        0           NaN      1
R-MMU-9603983.0.00  R-MMU-1564169        0           NaN      3
R-MMU-9603983.30.00 R-MMU-1602473        0           NaN      1
R-MMU-168412.0.00    R-MMU-389424        0           NaN      1
R-MMU-168412.30.00  R-MMU-9014672        0           NaN      2
R-MMU-708349.0.00   R-MMU-9014678        0           NaN      2
R-MMU-708349.30.00   R-MMU-162873        0           NaN      1
R-MMU-6798474.0.00  R-MMU-8878914        0           NaN      1
R-MMU-6798474.30.00 R-MMU-8858270        0           NaN      1
R-MMU-6798528.0.00  R-MMU-5605262        0           NaN      1
R-MMU-6798528.30.00  R-MMU-444620        0           NaN      2
R-MMU-9844860.0.00  R-MMU-9009950        0           NaN      1
R-MMU-9844860.30.00 R-MMU-8937744       

R-MMU-9816449.0.00  R-MMU-6804402        0           NaN      2
R-MMU-9816449.30.00  R-MMU-418172        0           NaN      2
R-MMU-8849857.0.00  R-MMU-9708866        0           NaN      1
R-MMU-8849857.30.00 R-MMU-9006144        0           NaN      1
R-MMU-1461971.0.00  R-MMU-5336430        0           NaN      2
R-MMU-1461971.30.00 R-MMU-5687126        0           NaN      1
R-MMU-1461982.0.00  R-MMU-9624527        0           NaN      1
R-MMU-1461982.30.00 R-MMU-9699581        0           NaN      1
R-MMU-1461995.0.00  R-MMU-5619439        0           NaN      1
R-MMU-1461995.30.00  R-MMU-417896        0           NaN      1
R-MMU-1462003.0.00  R-MMU-9634937        0           NaN      1
R-MMU-1462003.30.00  R-MMU-381608        0           NaN      1
R-MMU-1462005.0.00  R-MMU-8959510        0           NaN      1
R-MMU-1462005.30.00 R-MMU-9034714        0           NaN      2
R-MMU-1462014.0.00  R-MMU-9615249        0           NaN      1
R-MMU-1462014.30.00 R-MMU-3296326       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id030_time_cross_20.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id030_time_cross_20.txt",sep=""), header=FALSE)
print(sid)

           V1
1  SRR3593523
2  SRR7817700
3  SRR7817648
4  SRR7817672
5  SRR7817662
6  SRR7817616
7  SRR4317609
8  SRR1636591
9  SRR1636586
10 SRR7817695
11 SRR7817663
12 SRR7817635
13 SRR7817637
14 SRR4317658
15 SRR1636676
16 SRR7817679
17 SRR7817655
18 SRR7817657
19 SRR7817645
20 SRR7817703
21 SRR7817622
22 SRR7817666
23 SRR7817690
24 SRR7817633
25 SRR7817644
26 SRR3593580


In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) + 
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  
   scale_colour_manual(values = c("0" = "red", "30" = "green")) +  
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section), 
     colors = c("red", "green"), 
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  
)
saveWidget(p, "plotly_chart030_time_cross_20.html", selfcontained = TRUE)